This is the pipeline between iNaturalist and ArcGIS location platform

To do:
- Test with iNaturalist project instead of just downloading all butterfly from Singapore daily
- Test with ArcGIS Notebook to run script more frequently

In [1]:
import requests
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from datetime import date
import os
from dotenv import load_dotenv
from pyinaturalist import get_observations

In [2]:
load_dotenv()
username = os.getenv("ARCGIS_USER")
password = os.getenv("ARCGIS_PASS")

# 1. Connect to ArcGIS 
gis = GIS("https://www.arcgis.com", username, password)

# 2. Link the Butterfly Layer
layer_url = "https://services7.arcgis.com/NoGQYZi3a1tVAyzH/arcgis/rest/services/butterfly_data_for_arcGIS/FeatureServer/0"
butterfly_layer = FeatureLayer(layer_url)

# 3. Drawing from iNaturalist using Rest API
# today = date.today().isoformat()
# inat_url = f"https://api.inaturalist.org/v1/observations?place_id=6734&taxon_id=47224&d1={today}&per_page=200"
# --- THE SYNC LOGIC ---
# print("Fetching from iNaturalist...")
# observations = requests.get(inat_url).json()

# 4. Using pyinaturalist to get observations
project_id = 'butterfly-test'
observations = get_observations(project_id = project_id, page = 'all')
print(observations)

{'results': [{'quality_grade': 'casual', 'taxon_geoprivacy': None, 'annotations': [], 'uuid': 'e7c0de4c-10d6-4a33-bba4-3c9d79c8fd7f', 'observed_on_details': {'date': '2026-02-16', 'day': 16, 'month': 2, 'year': 2026, 'hour': 15, 'week': 8}, 'id': 339028277, 'cached_votes_total': 0, 'identifications_most_agree': False, 'created_at_details': {'date': '2026-02-17', 'day': 17, 'month': 2, 'year': 2026, 'hour': 21, 'week': 8}, 'species_guess': 'Common Tiger Butterfly', 'identifications_most_disagree': False, 'tags': [], 'positional_accuracy': 2, 'comments_count': 0, 'site_id': 1, 'created_time_zone': 'Asia/Singapore', 'license_code': 'cc-by-nc', 'observed_time_zone': 'Asia/Singapore', 'quality_metrics': [], 'public_positional_accuracy': 2, 'reviewed_by': [10002273], 'oauth_application_id': None, 'flags': [], 'created_at': datetime.datetime(2026, 2, 17, 21, 39, 11, tzinfo=tzoffset(None, 28800)), 'description': '', 'time_zone_offset': '+08:00', 'project_ids_with_curator_id': [], 'observed_on'

In [3]:
cleaned_features = []

for obs in observations['results']:
    # Handle the 'Confidence' and 'Transect' as blank/None
    # iNaturalist doesn't have a direct 'transect' field in standard obs
    # Get the username safely
    user_info = obs.get('user', {})
    inat_username = user_info.get('login', 'Unknown')    

    # Safely get taxon name
    taxon_name = obs.get('taxon', {}).get('preferred_common_name', "Unknown")
    
    # Extract date details safely
    date_details = obs.get('observed_on_details', {})
    
    feature = {
        "attributes": {
            "ID": None, # Leave blank 
            "iNat_ID" : str(obs['id']),
            "Username" : inat_username,
            "Year": date_details.get('year'),
            "Month": date_details.get('month'),
            "Park": obs.get('place_guess', "Singapore"),
            "Latitude": obs['geojson']['coordinates'][1],
            "Longitude": obs['geojson']['coordinates'][0],
            "Transect": None,  # Leave blank
            "Start_Time": obs.get('time_observed_at'),
            "Confidence": None, # Leave blank
            "Species": taxon_name,
            "Count_": 1
        },
        "geometry": {
            "x": obs['geojson']['coordinates'][0],
            "y": obs['geojson']['coordinates'][1],
            "spatialReference": {"wkid": 4326}
        }
    }
    cleaned_features.append(feature)

print(cleaned_features)

# Push the cleaned list to ArcGIS
if cleaned_features:
    butterfly_layer.edit_features(adds=cleaned_features)
    print(f"Successfully added {len(cleaned_features)} sightings!")

[{'attributes': {'ID': None, 'iNat_ID': '339028277', 'Username': 'zhiquanchng', 'Year': 2026, 'Month': 2, 'Park': 'Toa Payoh, Singapore', 'Latitude': 1.3405009268, 'Longitude': 103.8742038409, 'Transect': None, 'Start_Time': None, 'Confidence': None, 'Species': 'Common Tiger Butterfly', 'Count_': 1}, 'geometry': {'x': 103.8742038409, 'y': 1.3405009268, 'spatialReference': {'wkid': 4326}}}]
Successfully added 1 sightings!


Test API with observations from iNaturalist Project

Create draft visualisations with streamlit

Test hosting visualiastions on arcGIS public